# QLSD: Quantised Langevin stochastic dynamics for Bayesian federated learning

## Table of contents:
* [Bayesian logistic regression](#section-log)

    * [Figure 1 - Synthetic data](#subsection-synthetic)
    
    * [Figure 2 and Table 2 - Real data](#subsection-real)

#### Packages

In [4]:
import scipy.io
import json
import seaborn as sns
from tqdm import trange
import math
from torchvision.datasets import EMNIST
from torchvision.datasets import DatasetFolder
from torchvision import transforms
import numpy as np
import pandas as pd
import copy
%matplotlib inline
import matplotlib.pyplot as plt

#### Useful functions

In [2]:
def SQuantization(s,x): 
    
    # s : number of quantization levels
    # x : vector to compress
    
    if s == 0:
        return x
    norm_x = np.linalg.norm(x,2)
    if norm_x == 0:
        return x
    ratio = np.abs(x) / norm_x
    l = np.floor(ratio * s)
    p = ratio * s - l
    sampled = np.random.binomial(1,p)
    qtzt = np.sign(x) * norm_x * (l + sampled) / s
    return qtzt

def com_bits(s,d):
    if s <= np.sqrt(d/2 - np.sqrt(d)):
        return (3 + (3/2) * np.log(2*(s**2 + d)/(s*(s+np.sqrt(d))))) * s*(s+np.sqrt(d)) + 32
    elif s == np.sqrt(d):
        return 2.8*d + 32
    else:
        return ((1/2)*(np.log(1 + (s**2+np.minimum(d,s*np.sqrt(d)))/d) + 1) + 2) * d + 32

In [2]:
def softmax(x):
    ex = np.exp(x)
    sum_ex = np.sum( np.exp(x))
    return ex/sum_ex


def generate_synthetic(alpha, beta, iid, d, nb_class, b):
    
    
    np.random.seed(1994)
    #samples_per_user = np.random.lognormal(4, 2, (b)).astype(int) + 50
    samples_per_user = np.random.randint(low=10,high=50,size=b).astype(int)
    num_samples = np.sum(samples_per_user)

    X_split = [[] for _ in range(b)]
    y_split = [[] for _ in range(b)]


    #### define some eprior ####
    np.random.seed(1994)
    mean_W = np.random.normal(0, alpha, b)
    mean_b = mean_W
    np.random.seed(1994)
    B = np.random.normal(0, beta, b)
    mean_x = np.zeros((b, d))

    diagonal = np.zeros(d)
    for j in range(d):
        diagonal[j] = np.power((j+1), -1.2)
    cov_x = np.diag(diagonal)

    for i in range(b):
        if iid == 1:
            mean_x[i] = np.ones(d) * B[i]  # all zeros
        else:
            np.random.seed(1994)
            mean_x[i] = np.random.normal(B[i], 1, d)

    if iid == 1:
        np.random.seed(1994)
        W_global = np.random.normal(0, 1, (d, nb_class))
        np.random.seed(1994)
        b_global = np.random.normal(0, 1,  nb_class)

    for i in range(b):
        np.random.seed(1994)
        W = np.random.normal(mean_W[i], 1, (d, nb_class))
        np.random.seed(1994)
        b = np.random.normal(mean_b[i], 1,  nb_class)

        if iid == 1:
            W = W_global
            b = b_global
        np.random.seed(1994)
        xx = np.random.multivariate_normal(mean_x[i], cov_x, samples_per_user[i])
        yy = np.zeros(samples_per_user[i])

        for j in range(samples_per_user[i]):
            tmp = np.dot(xx[j], W) + b
            yy[j] = np.argmax(softmax(tmp))

        X_split[i] = xx.tolist()
        y_split[i] = yy.tolist()

    return X_split, y_split

## 2. Bayesian logistic regression  <a class="anchor" id="section-log"></a>

#### Algorithms

In [ ]:
def MALA(T_total,T_bi,T,gamma,init):
    
    # Definition of the arrays of interest
    loss = np.zeros(T)
    acc = np.zeros(T)
    theta = np.zeros((T,d))
    
    # Initialisation
    theta[0,:] = init
    x = theta[0,:]
    P = 1 / (1 + np.exp(-np.dot(X_tot,theta[0,:])))
    P = P - 1e-5
    P[P<0] = 1e-10
    loss[0] = - np.dot(y_tot,np.log(P)) - np.dot(1-y_tot,np.log(1-P)) + (tau/2) * np.linalg.norm(x)**2
    
    # Parameters
    thin = int((T_total-T_bi+1)/T)

    # MCMC sampler    
    for t in range(T_total-1): # total number of iterations
        
        P = 1 / (1 + np.exp(-np.dot(X_tot,x)))
        grad = np.dot(X_tot.T, np.reshape(P - y_tot,(np.sum(n),))) + tau * x
        grad = np.reshape(grad,(d,))
        s = np.zeros(d)
        s = x - gamma * grad + np.sqrt(2*gamma) * np.random.normal(0,1,d)
    
        u = np.random.uniform()
        P = 1 / (1 + np.exp(-np.dot(X_tot,s)))
        U = - np.dot(y_tot,np.log(P)) - np.dot(1-y_tot,np.log(1-P)) + tau/2 * np.linalg.norm(s)**2
        P = 1 / (1 + np.exp(-np.dot(X_tot,x)))
        U_old = - np.dot(y_tot,np.log(P)) - np.dot(1-y_tot,np.log(1-P)) + tau/2 * np.linalg.norm(x)**2
        P = 1 / (1 + np.exp(-np.dot(X_tot,s)))
        grad_prop = np.dot(X_tot.T, np.reshape(P - y_tot,(np.sum(n),))) + tau * s
        grad_prop = np.reshape(grad,(d,))
        ratio = np.exp(-U + U_old \
                       - (1/(4*gamma))*np.linalg.norm(x - s + gamma * grad_prop,2)**2 \
                       + (1/(4*gamma))*np.linalg.norm(s - x + gamma * grad,2)**2)
        #print(min(1,ratio))
        if u <= ratio:
            x = s
          
        # Save theta according to the thinning strategy
        if (t % thin == 0) and (t >= T_bi):
            theta[int((t-T_bi)/thin) + 1,:] = x
            P = 1 / (1 + np.exp(-np.dot(X_tot,x)))
            P = P - 1e-5
            P[P<0] = 1e-10
            loss[int((t-T_bi)/thin) + 1] = - np.dot(y_tot,np.log(P)) - np.dot(1-y_tot,np.log(1-P)) + (tau/2) * np.linalg.norm(x)**2
            
            # Accuracy
            a = np.exp(X_tot.dot(x)) / (1 + np.exp(X_tot.dot(x)))
            a[a >= 0.5] = 1
            a[a < 0.5] = 0
            err = np.sum(np.abs(a - y_tot))/len(y_tot)
            acc[int((t-T_bi)/thin) + 1] = 1 - err
        
        if t % 100000 == 0:
            print(t)
            
    return theta, loss, acc

def QLSDpp(T_total,T_bi,T,gamma,p,init,s,K,beta,memory):
    
    # Definition of the arrays of interest
    loss = np.zeros(T)
    theta = np.zeros((T,d))
    qgrad = np.zeros((b,d))
    h = np.zeros((b,d))
    
    # Initialisation
    theta[0,:] = init
    x = init
    P = 1 / (1 + np.exp(-np.dot(X_tot,theta[0,:])))
    P = P - 1e-5
    P[P<0] = 1e-10
    loss[0] = - np.dot(y_tot,np.log(P)) - np.dot(1-y_tot,np.log(1-P)) + (tau/2) * np.linalg.norm(theta[0,:])**2
    z = theta[0,:]
    
    # Parameters
    thin = int((T_total-T_bi+1)/T)

    # MCMC sampler    
    for t in range(T_total-1): # total number of iterations
        
        if t % K == 0: # update of the control variate every K iterations
            z = x

        for i in range(b): # for each client in parallel
            
             # Compute full gradient at control variate
            P = 1 / (1 + np.exp(-np.dot(X[i],z)))
            fullgrad_z = -np.dot(X[i].T, np.reshape(y[i] - P,(n[i],))) + tau * n[i]/np.sum(n) * z
            fullgrad_z = np.reshape(fullgrad_z,(d,))

            # Compute stochastic gradient based on SVRG scheme
            idx = np.random.randint(0, n[i])
            grad_theta = (1 / (1 + np.exp(-np.dot(X[i][idx,:],x))) - y[i][idx]) * X[i][idx,:] \
                         + tau * (1/np.sum(n)) * x
            grad_z = (1 / (1 + np.exp(-np.dot(X[i][idx,:],z))) - y[i][idx]) * X[i][idx,:] \
                         + tau * (1/np.sum(n)) * z
            grad = n[i] * (grad_theta - grad_z) + fullgrad_z

            # Quantize the stochastic gradient minus memory term
            qqgrad = SQuantization(s, grad - h[i,:])
            qgrad[i,:] = qqgrad + h[i,:]
            if memory == True:
                h[i,:] = h[i,:] + beta * qqgrad   
        
        
        # Update theta on server
        x = x - gamma * np.sum(qgrad,axis=0) + np.sqrt(2*gamma) * np.random.normal(0,1,size=d)
    
        # Save theta according to the thinning strategy
        if (t % thin == 0) and (t >= T_bi):
            theta[int((t-T_bi)/thin) + 1,:] = x
            P = 1 / (1 + np.exp(-np.dot(X_tot,x)))
            P = P - 1e-5
            P[P<0] = 1e-10
            loss[int((t-T_bi)/thin) + 1] = - np.dot(y_tot,np.log(P)) - np.dot(1-y_tot,np.log(1-P)) \
                                           + (tau/2) * np.linalg.norm(x)**2
            
    return theta, loss

def LSDpp(T_total,T_bi,T,gamma,p,init,K):
    
    # Definition of the arrays of interest
    loss = np.zeros(T)
    theta = np.zeros((T,d))
    grad = np.zeros((b,d))
    
    # Initialisation
    theta[0,:] = init
    x = init
    P = 1 / (1 + np.exp(-np.dot(X_tot,theta[0,:])))
    P = P - 1e-5
    P[P<0] = 1e-10
    loss[0] = - np.dot(y_tot,np.log(P)) - np.dot(1-y_tot,np.log(1-P)) + (tau/2) * np.linalg.norm(theta[0,:])**2
    z = theta[0,:]
    
    # Parameters
    thin = int((T_total-T_bi+1)/T)

    # MCMC sampler    
    for t in range(T_total-1): # total number of iterations
        
        if t % K == 0: # update of the control variate every K iterations
            z = x

        for i in range(b): # for each client in parallel
            
             # Compute full gradient at control variate
            P = 1 / (1 + np.exp(-np.dot(X[i],z)))
            fullgrad_z = -np.dot(X[i].T, np.reshape(y[i] - P,(n[i],))) + tau * n[i]/np.sum(n) * z
            fullgrad_z = np.reshape(fullgrad_z,(d,))

            # Compute stochastic gradient based on SVRG scheme
            idx = np.random.randint(0, n[i])
            grad_theta = (1 / (1 + np.exp(-np.dot(X[i][idx,:],x))) - y[i][idx]) * X[i][idx,:] \
                         + tau * (1/np.sum(n)) * x
            grad_z = (1 / (1 + np.exp(-np.dot(X[i][idx,:],z))) - y[i][idx]) * X[i][idx,:] \
                         + tau * (1/np.sum(n)) * z
            grad[i,:] = n[i] * (grad_theta - grad_z) + fullgrad_z 
        
        
        # Update theta on server
        x = x - gamma * np.sum(grad,axis=0) + np.sqrt(2*gamma) * np.random.normal(0,1,size=d)
    
        # Save theta according to the thinning strategy
        if (t % thin == 0) and (t >= T_bi):
            theta[int((t-T_bi)/thin) + 1,:] = x
            P = 1 / (1 + np.exp(-np.dot(X_tot,x)))
            P = P - 1e-5
            P[P<0] = 1e-10
            loss[int((t-T_bi)/thin) + 1] = - np.dot(y_tot,np.log(P)) - np.dot(1-y_tot,np.log(1-P)) \
                                           + (tau/2) * np.linalg.norm(x)**2
            
    return theta, loss

### 2.1. Synthetic data <a class="anchor" id="subsection-synthetic"></a>

#### Generate data

In [ ]:
b = 20 # number of clients
nb_class = 2 # number of classes
d = 10 # dimension

train_data = {'users': [], 'user_data':{}, 'num_samples':[]}
test_data = {'users': [], 'user_data':{}, 'num_samples':[]}


X, y = generate_synthetic(alpha=1, beta=1, iid=0, d=d, nb_class=nb_class, b=b)     # synthetic (1,1)


# Create data structure
train_data = {'users': [], 'user_data':{}, 'num_samples':[]}
test_data = {'users': [], 'user_data':{}, 'num_samples':[]}
    
for i in trange(b, ncols=120):

    uname = 'f_{0:05d}'.format(i)        
    combined = list(zip(X[i], y[i]))
    np.random.shuffle(combined)
    X[i][:], y[i][:] = zip(*combined)
    num_samples = len(X[i])
    train_len = int(0.9 * num_samples)
    test_len = num_samples - train_len
        
    train_data['users'].append(uname) 
    train_data['user_data'][uname] = {'x': X[i][:train_len], 'y': y[i][:train_len]}
    train_data['num_samples'].append(train_len)
    test_data['users'].append(uname)
    test_data['user_data'][uname] = {'x': X[i][train_len:], 'y': y[i][train_len:]}
    test_data['num_samples'].append(test_len)

In [15]:
# Define the observations vector y
y = []
for i in range(b):
    a = np.reshape(train_data['user_data'][train_data['users'][i]]['y'],(n[i],))
    a[a == -1] = 0
    y.append(a)
    
# Define the observation matrix X
X = []
for i in range(b):
    a = np.reshape(train_data['user_data'][train_data['users'][i]]['x'],(n[i],d))
    X.append(a)
    
# Define total observations and features
y_tot = np.concatenate(y,axis=0)
X_tot = np.concatenate(X,axis=0)

#### Launch algorithms

In [17]:
w, v = np.linalg.eig(X_tot.T.dot(X_tot))
w = np.max(w.real)
tau = 1000
M = w/4 + tau
S = [2**1,2**2,2**4]
K = 100
gamma = 1/(M)
omega = [np.minimum(d/s**2,np.sqrt(d)/s) for s in S]
T = 100000

thetaMALA, lossMALA, acc = MALA(T_total=T,T_bi=0,T=T,gamma=1/(10*M),init=np.zeros(d))

# Repetitions
rep = 20

# Init.
theta1 = np.zeros((rep,T,d))
theta2 = np.zeros((rep,T,d))
theta3 = np.zeros((rep,T,d))
theta4 = np.zeros((rep,T,d))
theta5 = np.zeros((rep,T,d))
theta6 = np.zeros((rep,T,d))
theta7 = np.zeros((rep,T,d))

for r in range(rep):
    
    np.random.seed(r)

    theta1[r,:],loss1 = QLSDpp(T_total=T,T_bi=0,T=T,gamma=gamma,p=0.1,init=np.zeros(d),s=S[0],K=K,beta=1/(omega[0]+1),memory=True)
    theta2[r,:],loss2 = QLSDpp(T_total=T,T_bi=0,T=T,gamma=gamma,p=0.1,init=np.zeros(d),s=S[0],K=K,beta=1/(omega[0]+1),memory=False)
    theta3[r,:],loss3 = QLSDpp(T_total=T,T_bi=0,T=T,gamma=gamma,p=0.1,init=np.zeros(d),s=S[1],K=K,beta=1/(omega[1]+1),memory=True)
    theta4[r,:],loss4 = QLSDpp(T_total=T,T_bi=0,T=T,gamma=gamma,p=0.1,init=np.zeros(d),s=S[1],K=K,beta=1/(omega[1]+1),memory=False)
    theta5[r,:],loss5 = QLSDpp(T_total=T,T_bi=0,T=T,gamma=gamma,p=0.1,init=np.zeros(d),s=S[2],K=K,beta=1/(omega[2]+1),memory=True)
    theta6[r,:],loss6 = QLSDpp(T_total=T,T_bi=0,T=T,gamma=gamma,p=0.1,init=np.zeros(d),s=S[2],K=K,beta=1/(omega[2]+1),memory=False)
    theta7[r,:],loss7 = LSDpp(T_total=T,T_bi=0,T=T,gamma=gamma,p=0.1,init=np.zeros(d),K=K)
    
    print(r)

0
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


### 2.2. Real data <a class="anchor" id="subsection-real"></a>

#### Generate data

In [5]:
train_dataset = EMNIST('./data', split="digits", train=True, download=True,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))]))
test_dataset = EMNIST('./data', split="digits", train=False, transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))]))

def femnist_star(dataset, num_users):
    """
        Sample non-IID client data from EMNIST dataset -> FEMNIST*
        :param dataset:
        :param num_users:
        :return: dict of image index
    """
    print("Sampling dataset: FEMNIST*")
    np.random.seed(1994)

    dict_users = {i: [] for i in range(num_users)}
    total_len = len(dataset)

    labels = dataset.targets.numpy()
    idxs = np.argsort(labels)

    num_shards, num_imgs = 26 * num_users, total_len // (num_users * 26)

    label_selected = [np.random.choice(26, 20, replace=False) for _ in range(num_users)]

    label_selected_1 = [np.random.choice(label_selected[i], 6, replace=False) for i in range(num_users)]
    for i in range(num_users):
        for j in label_selected[i]:
            ind_pos = np.random.choice(num_users)
            tmp = copy.deepcopy(idxs[j * num_users * num_imgs + ind_pos * num_imgs: j * num_users * num_imgs + (ind_pos + 1) * num_imgs])
            dict_users[i].append(tmp)
        for j in label_selected_1[i]:
            ind_pos = np.random.choice(num_users)
            tmp = copy.deepcopy(idxs[j * num_users * num_imgs + ind_pos * num_imgs: j * num_users * num_imgs + (
                        ind_pos + 1) * num_imgs])
            dict_users[i].append(tmp)

    for i in range(num_users):
        dict_users[i] = np.concatenate(tuple(dict_users[i]), axis=0)
    return dict_users

#### Parameters

In [6]:
b = 10 # number of clients
n_max = 100 # maximum number of observations on each client
n_min = 20 # minimum number of observations on each client
np.random.seed(1994)
n = np.random.randint(low=n_min,high=n_max,size=b) # number of observations per client

d = 28*28

dict_users = femnist_star(train_dataset,b)
X = []
y = []
for i in range(b):
    np.random.seed(1994)
    idx_client = np.random.choice(dict_users[i],n[i],replace=False)
    X.append([np.reshape(train_dataset[i][0].numpy(),(d,)) for i in idx_client])
    y.append([train_dataset[i][1] for i in idx_client])

# Define total observations and features
y_tot = np.concatenate(y,axis=0)
X_tot = np.concatenate(X,axis=0)

#### Launch algorithms

In [ ]:
# Parameters
K = 10
tau = 100
w, v = np.linalg.eig(np.dot(X_tot.T,X_tot))
w = np.max(w.real)
M = w * (1/4) + tau 
gamma = 1/(5*M)
K = 100
p = 0.2
S = [2**4,2**8,2**16]
omega = [np.minimum(d/s**2,np.sqrt(d)/s) for s in S]
init = np.random.normal(0,1,size=d)
T = 500000

theta1,loss1,acc1 = LSDpp(T_total=T,T_bi=0,T=T,gamma=gamma,p=p,init=init,L=L)
theta2,loss2,acc2 = QLSDpp(T_total=T,T_bi=0,T=T,gamma=gamma,p=p,init=init,s=S[0],K=K,alpha=1/(omega[1]+1))
theta3,loss3,acc3 = QLSDpp(T_total=T,T_bi=0,T=T,gamma=gamma,p=p,init=init,s=S[1],K=K,alpha=1/(omega[2]+1))
theta4,loss4,acc4 = QLSDpp(T_total=T,T_bi=0,T=T,gamma=gamma,p=p,init=init,s=S[2],K=K,alpha=1/(omega[3]+1))

#### HPD error

In [ ]:
N = 200
Tbi = 50000
alpha = 0.01
beta = 0.99
eta1 = np.zeros(N)
eta2 = np.zeros(N)
eta3 = np.zeros(N)
eta4 = np.zeros(N)

for i in range(N):
    eta1[i] = np.quantile(loss1[Tbi:],np.linspace(alpha,beta,N)[i])
    eta2[i] = np.quantile(loss2[Tbi:],np.linspace(alpha,beta,N)[i])
    eta3[i] = np.quantile(loss3[Tbi:],np.linspace(alpha,beta,N)[i])
    eta4[i] = np.quantile(loss4[Tbi:],np.linspace(alpha,beta,N)[i])
    
print((np.abs(eta2-eta1)/eta1)[99])
print((np.abs(eta3-eta1)/eta1)[99])
print((np.abs(eta4-eta1)/eta1)[99])

#### Predictive distribution on test data

In [ ]:
X = [np.reshape(test_dataset[i][0].numpy(),(d,)) for i in range(1000)]
y = [test_dataset[i][1] for i in range(1000)]

idx = [0,1,2,31,5,11,15,17]

T = 100000
Tbi = 500000 - T 
K = 10
probaLSD = np.zeros((len(idx),T,K))
probaQLSD = np.zeros((len(idx),T,K))
for t in range(T):
    for i in range(len(idx)):
        AQLSD = np.reshape(np.exp(np.dot(X[idx[i]],theta2[Tbi+t,:])),(1,K))
        ALSD = np.reshape(np.exp(np.dot(X[idx[i]],theta1[Tbi+t,:])),(1,K))
        for k in range(K):
            probaQLSD[i,t,k] = np.exp(np.dot(X[idx[i]],theta2[Tbi+t,:,k])) / np.sum(AQLSD,axis=1)
            probaLSD[i,t,k] = np.exp(np.dot(X[idx[i]],theta1[Tbi+t,:,k])) / np.sum(ALSD,axis=1)
            
indicesMax = np.zeros(len(idx))
for i in range(len(idx)):
    indicesMax[i] = np.reshape(np.where(np.mean(probaQLSD[i,:],axis=0) \
                                        == np.max(np.mean(probaQLSD[i,:],axis=0))),(1,))

# QLSD
df = pd.DataFrame(data=probaQLSD[0,:,int(indicesMax[0])],
                  columns=['Predictive distribution of most probable label'])
df['Algorithm'] = 'QLSD++ 4 bits'
df['observation'] = 0
for i in range(len(idx)-1):
    i += 1
    df1 = pd.DataFrame(data=probaQLSD[i,:,int(indicesMax[i])],
                       columns=['Predictive distribution of most probable label'])
    df1['Algorithm'] = 'QLSD++ 4 bits'
    df1['observation'] = i
    df = pd.concat([df,df1])

dfLSD = pd.DataFrame(data=probaLSD[0,:,int(indicesMax[0])],
                     columns=['Predictive distribution of most probable label'])
dfLSD['Algorithm'] = 'LSD++'
dfLSD['observation'] = 0
for i in range(len(idx)-1):
    i += 1
    df1 = pd.DataFrame(data=probaLSD[i,:,int(indicesMax[i])],
                       columns=['Predictive distribution of most probable label'])
    df1['Algorithm'] = 'LSD++'
    df1['observation'] = i
    dfLSD = pd.concat([dfLSD,df1])

df = pd.concat([df,dfLSD])

In [ ]:
plt.figure()
g = sns.boxplot(x = df['observation'], 
            y = df['Predictive distribution of most probable label'],
            hue = df['Algorithm'],
            whis=[5, 95])
g.set(xticklabels=[])  
g.set(xlabel=None)
plt.legend(bbox_to_anchor=(0.1, 0.35), loc=2, borderaxespad=0.);